In [80]:
import warnings 
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import datetime as dt
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
pd.set_option('display.max_columns', None)
# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Background/Ideas

- Features will be based off seasonal,weekly, and career based values
- Idea is that certain players which can be differentiated by career based values paired with weekly performance and or seasonal (team strength proxy) can be paired to build something relatively predictive.
- Interactivity can be dependent on clicking and choosing assortment of players and identifying/projecting current projections.
- Data seems to get updated weekly so these predictions would change over time as well.

# Data

This section focuses on pulling the data and prepping/aggregating the dependent variable. (Fantasy Points)

In [2]:
roster_data = nfl.import_seasonal_rosters([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999])
pbp_df = pd.DataFrame(nfl.import_pbp_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
# injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000]))
schedules_df = pd.DataFrame(nfl.import_schedules([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

2024 done.
2023 done.
2022 done.
2021 done.
2020 done.
2019 done.
2018 done.
2017 done.
2016 done.
2015 done.
2014 done.
2013 done.
2012 done.
2011 done.
2010 done.
2009 done.
2008 done.
2007 done.
2006 done.
2005 done.
2004 done.
2003 done.
2002 done.
2001 done.
2000 done.
1999 done.
Downcasting floats.
Downcasting floats.


# Basic Player Related Stats 

**Game by Game**

0                None
1                None
2                None
3          00-0035228
4          00-0035228
              ...    
1202652    00-0011024
1202653    00-0011024
1202654          None
1202655    00-0011024
1202656          None
Name: passer_player_id, Length: 1202657, dtype: object

In [35]:
team['depth_chart_position'].unique()

array(['T', 'QB', 'P', 'K', 'TE', 'LS', 'DE', 'SS', 'NT', 'DT', 'C',
       'ILB', 'FS', 'CB', 'G', 'MLB', 'FB', 'WR', 'RB', 'OLB', 'DB', 'LB',
       'OT', 'OG', 'S', 'SAF', 'HB', 'DL', None, 'OL', 'PR'], dtype=object)

In [36]:
## Basic PBP Passing Stats

def get_opposing_team(df):
    if df['home_team'] == df['team']:
        val = df['away_team']
    elif df['away_team'] == df['team']:
        val = df['home_team']
    else:
        val = None

    return val

passing_stats = pbp_df[~pbp_df['passer_player_id'].isna()].copy()

passing_stats = pbp_df.groupby(['game_id', 'game_date','season', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'passer_player_id', 'passer_player_name']).agg({
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum',
    'interception': 'sum',
    'was_pressure': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'fumble': 'sum'
}).reset_index()


## Grabbing seasonal info

team = roster_data[roster_data['depth_chart_position'] == 'QB'][['season','player_id','team','depth_chart_position']]

team['team'] = team['team'].replace({'OAK':'LV', 'STL':'LA', 'SD':'LAC','HST':'HOU', 'BLT':'BAL', 'CLV':'CLE','SL':'LA','ARZ':'ARI'})


team.rename(columns = {'player_id':'passer_player_id'},inplace = True)

passing_stats = passing_stats.merge(team, on = ['passer_player_id','season'], how = 'inner')


## Aggregate average score to opposition 

passing_stats['opponent_team'] = passing_stats.apply(get_opposing_team,axis = 1)

print('Number Missing Opponent:' + str(passing_stats[passing_stats['opponent_team'].isna()].shape[0]))
passing_stats = passing_stats[~passing_stats['opponent_team'].isna()]

home_teams = schedules_df[['season','home_team','away_score']].copy()

away_teams = schedules_df[['season','away_team','home_score']].copy()

home_teams.rename(columns = {'home_team':'team','away_score':'points_allowed'}, inplace = True)
away_teams.rename(columns = {'away_team':'team','home_score':'points_allowed'}, inplace = True)

points_allowed_df = pd.concat([home_teams,away_teams])


points_allowed_df['avg_points_allowed'] = points_allowed_df.groupby(['season','team'])['points_allowed'].transform('mean')


points_allowed_df.rename(columns = {'team':'opponent_team'},inplace = True)


kicker_stats = passing_stats.merge(points_allowed_df[['season','opponent_team','avg_points_allowed']].drop_duplicates(), on = ['opponent_team','season'], how = 'left')

#Checking the passing stats dataframe
passing_stats.head(2)

Number Missing Opponent:16


,game_id,game_date,season,week,div_game,home_team,away_team,weather,location,stadium,spread_line,total_line,roof,surface,temp,wind,home_coach,away_coach,passer_player_id,passer_player_name,pass_attempt,complete_pass,passing_yards,air_yards,pass_touchdown,interception,was_pressure,rush_attempt,rushing_yards,rush_touchdown,lateral_rush,fumble,team,depth_chart_position,opponent_team
0,2001_01_ATL_SF,2001-09-09,2001,1,1,SF,ATL,"partly cloudy Temp: 68° F, Humidity: 63%, Wind...",Home,3COM Park,3.5,46.0,outdoors,grass,68.0,12.0,Steve Mariucci,Dan Reeves,00-0002876,C.Chandler,20.0,11.0,121.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,ATL,QB,SF
1,2001_03_ATL_ARI,2001-09-30,2001,3,0,ARI,ATL,"Temp: 104° F, Humidity: 13%, Wind: East 20 mph",Home,Sun Devil Stadium,-3.0,42.0,outdoors,grass,104.0,20.0,Dave McGinnis,Dan Reeves,00-0002876,C.Chandler,30.0,20.0,286.0,0.0,3.0,0.0,0,0.0,0.0,0.0,0.0,0.0,ATL,QB,ARI


**Expanding it to Seasonal**

- Avg Passing Touchdowns per game
- Avg Interceptions per game
- Avg Rush Attempts
- Average Offensive Snaps Facing Pressure
- Proportion of Rush Touchdowns to Passing Touchdowns
- Average Rushing Touchdowns per game
- Average Rushing Yards 
- Average Passing Yards

In [61]:

## Avg Passing TD's
passing_stats['avg_ptd'] = passing_stats.groupby(['passer_player_id','season'])['pass_touchdown'].transform('mean')

## Avg Interceptions
passing_stats['avg_int'] = passing_stats.groupby(['passer_player_id','season'])['interception'].transform('mean')

## Avg Rush Attempts

passing_stats['avg_ra'] = passing_stats.groupby(['passer_player_id','season'])['rush_attempt'].transform('mean')


## Average Offensive Snaps Facing Pressure


passing_stats['avg_pressure_snaps'] = passing_stats.groupby(['passer_player_id','season'])['was_pressure'].transform('mean')


## Prop of Rush TD to Pass TD
passing_stats['total_rush_td'] = passing_stats.groupby(['passer_player_id','season'])['rush_touchdown'].transform('sum')
passing_stats['total_pass_td'] = passing_stats.groupby(['passer_player_id','season'])['pass_touchdown'].transform('sum')

passing_stats['rush_to_pass'] = passing_stats['total_rush_td'] / passing_stats['total_pass_td']


## Average Rushing TD's per game

passing_stats['avg_rush_td'] = passing_stats.groupby(['passer_player_id','season'])['rush_touchdown'].transform('mean')



## Avg Rushing Yards

passing_stats['avg_rush_yd'] = passing_stats.groupby(['passer_player_id','season'])['rushing_yards'].transform('mean')



## Avg Passing Yards

passing_stats['avg_pass_yd'] = passing_stats.groupby(['passer_player_id','season'])['passing_yards'].transform('mean')


## Avg Yards Per Pass Attempt

passing_stats['total_pass_attempts'] = passing_stats.groupby(['passer_player_id','season'])['pass_attempt'].transform('sum')
passing_stats['total_pass_yards'] = passing_stats.groupby(['passer_player_id','season'])['passing_yards'].transform('sum')
passing_stats['avg_yd_pass_att'] = passing_stats['total_pass_yards'] / passing_stats['total_pass_attempts']


## Completion Percentage
passing_stats['completion_percentage'] = passing_stats['complete_pass']/passing_stats['pass_attempt'] 


## Seasonal Completion Percentage

passing_stats['season_completion_percentage'] = passing_stats.groupby(['passer_player_id','season'])['completion_percentage'].transform('mean')

## Scorable Snaps (Scoring Opportunities)

passing_stats['scorable_snaps'] = passing_stats['rush_attempt'] + passing_stats['pass_attempt'] 

## Avg Scoreable Snaps 

passing_stats['avg_scorable_snaps'] = passing_stats.groupby(['passer_player_id','season'])['scorable_snaps'].transform('mean') 

## Games played since 1999


passing_stats['games_played_1999'] = passing_stats.groupby(['passer_player_id'])['game_id'].transform('count')

## Dependent Variable (Fantasy Points)

- Passing yards: 1 point for every 20–25 passing yards, or 0.04–0.05 points per passing yard
- Passing touchdowns: 4 points
- Rushing yards: 1 point for every 10 rushing yards
- Rushing touchdowns: 6 points 
- Other points that can be awarded include:
- Interceptions or fumbles lost -2 points
- Extra point: 1 point
- Field goal from 0–39 yards: 3 points
- Field goal from 40–49 yards: 4 points

In [54]:
passing_stats['fantasy_points'] = (( passing_stats['passing_yards'] * .05 ) + 
(passing_stats['pass_touchdown'] * 4) + 
(passing_stats['rushing_yards'] *.01) + (passing_stats['fumble'] * -2) + (passing_stats['interception'] * -2) + (passing_stats['rush_touchdown'] * 6)) 

In [56]:
passing_stats.sort_values('season', ascending = False)['fantasy_points'].head(2)

3903    0.0
3775    5.3
Name: fantasy_points, dtype: float32

# Team Related Stats

**Game By Game**

- Proportion of Plays that are Runs
- Offensive Snaps Ratio (Does the team spend more time on defense or offense)

In [48]:
## Average Points Allowed From QB 

passing_stats['opp_avg_qb_pts_allowed'] = passing_stats.groupby(['opponent_team','season'])['fantasy_points'].transform('mean')


## Average Completion Percentage Allowed

passing_stats['opp_pass_compl_percentage'] = passing_stats.groupby(['opponent_team','season'])['completion_percentage'].transform('mean')





# External Game Factor Stats

In [62]:
## Home Away Flag 



passing_stats['home_flag'] = np.where(passing_stats['opponent_team'] == passing_stats['home_team'], 0,1)
passing_stats['surface'] = passing_stats['surface'].str.strip()
passing_stats = passing_stats.join(pd.get_dummies(data = passing_stats['surface'], prefix = 'flag'))


passing_stats = passing_stats.join(pd.get_dummies(data = passing_stats['surface'], prefix = 'flag'))



## Interaction Vars with Wind and Temp on scorable attempts


passing_stats['wind_interaction'] = passing_stats['wind'] * passing_stats['scorable_snaps']
passing_stats['wind_interaction'] = passing_stats['wind_interaction'].fillna(0)

passing_stats['temp_interaction'] = passing_stats['temp'] * passing_stats['scorable_snaps']
passing_stats['temp_interaction'] = kicker_stats['temp_interaction'].fillna(0)



## Stadium 

ValueError: columns overlap but no suffix specified: Index(['flag_a_turf', 'flag_astroturf', 'flag_fieldturf', 'flag_grass'], dtype='object')

In [67]:
passing_stats.head(3)

,game_id,game_date,season,week,div_game,home_team,away_team,weather,location,stadium,spread_line,total_line,roof,surface,temp,wind,home_coach,away_coach,passer_player_id,passer_player_name,pass_attempt,complete_pass,passing_yards,air_yards,pass_touchdown,interception,was_pressure,rush_attempt,rushing_yards,rush_touchdown,lateral_rush,fumble,team,depth_chart_position,opponent_team,avg_ptd,avg_int,avg_ra,avg_pressure_snaps,total_rush_td,total_pass_td,rush_to_pass,avg_rush_td,avg_rush_yd,avg_pass_yd,total_pass_attempts,total_pass_yards,avg_yd_pass_att,fantasy_points,opp_avg_qb_pts_allowed,completion_percentage,opp_pass_compl_percentage,scorable_snaps,avg_scorable_snaps,season_completion_percentage,home_away_flag,flag_a_turf,flag_astroturf,flag_fieldturf,flag_grass,wind_interaction,games_played_1999
0,2001_01_ATL_SF,2001-09-09,2001,1,1,SF,ATL,"partly cloudy Temp: 68° F, Humidity: 63%, Wind...",Home,3COM Park,3.5,46.0,outdoors,grass,68.0,12.0,Steve Mariucci,Dan Reeves,00-0002876,C.Chandler,20.0,11.0,121.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,ATL,QB,SF,1.0,0.6,0.0,0.0,0.0,5.0,0.0,0.0,0.0,188.399994,146.0,942.0,6.452055,6.050000,9.815625,0.550000,0.507598,20.0,29.200001,0.559757,0,0,0,0,1,240.0,5
1,2001_03_ATL_ARI,2001-09-30,2001,3,0,ARI,ATL,"Temp: 104° F, Humidity: 13%, Wind: East 20 mph",Home,Sun Devil Stadium,-3.0,42.0,outdoors,grass,104.0,20.0,Dave McGinnis,Dan Reeves,00-0002876,C.Chandler,30.0,20.0,286.0,0.0,3.0,0.0,0,0.0,0.0,0.0,0.0,0.0,ATL,QB,ARI,1.0,0.6,0.0,0.0,0.0,5.0,0.0,0.0,0.0,188.399994,146.0,942.0,6.452055,26.299999,14.723530,0.666667,0.608467,30.0,29.200001,0.559757,0,0,0,0,1,600.0,5
2,2001_10_ATL_GB,2001-11-18,2001,10,0,GB,ATL,"Partly sunny, light rain possible. Temp: 59° F...",Home,Lambeau Field,10.0,39.5,outdoors,grass,59.0,16.0,Mike Sherman,Dan Reeves,00-0002876,C.Chandler,53.0,29.0,352.0,0.0,2.0,2.0,0,0.0,0.0,0.0,0.0,1.0,ATL,QB,GB,1.0,0.6,0.0,0.0,0.0,5.0,0.0,0.0,0.0,188.399994,146.0,942.0,6.452055,19.600000,8.732353,0.547170,0.506209,53.0,29.200001,0.559757,0,0,0,0,1,848.0,5


# Injury Related Variables

# List of Current Features

x_Vars: 
- div_game
- home_flag (Home if 1)
- spread_line
- pass_attempt
- complete_pass
- passing_yards
- air_yards
- pass_touchdown
- interception
- was_pressure
- rush_attempt
- rushing_yards
- rush_touchdown
- fumble
- avg_pressure_snaps
- avg_rush_td
- avg_rush_yd
- avg_pass_yd
- avg_yd_pass_att
- opp_avg_qb_pts_allowed
- completion_percentage
- opp_pass_compl_percentage
- scorable_snaps
- avg_scorable_snaps
- season_completion_percentage
- flag_a_turf
- flag_astrotuf
- flag_fieldturf
- flag_grass
- wind_interaction
- temp_interaction
- games_played_1999

Y_var:

- fantasy_points

In [68]:
passing_stats.columns

Index(['game_id', 'game_date', 'season', 'week', 'div_game', 'home_team',
       'away_team', 'weather', 'location', 'stadium', 'spread_line',
       'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach',
       'away_coach', 'passer_player_id', 'passer_player_name', 'pass_attempt',
       'complete_pass', 'passing_yards', 'air_yards', 'pass_touchdown',
       'interception', 'was_pressure', 'rush_attempt', 'rushing_yards',
       'rush_touchdown', 'lateral_rush', 'fumble', 'team',
       'depth_chart_position', 'opponent_team', 'avg_ptd', 'avg_int', 'avg_ra',
       'avg_pressure_snaps', 'total_rush_td', 'total_pass_td', 'rush_to_pass',
       'avg_rush_td', 'avg_rush_yd', 'avg_pass_yd', 'total_pass_attempts',
       'total_pass_yards', 'avg_yd_pass_att', 'fantasy_points',
       'opp_avg_qb_pts_allowed', 'completion_percentage',
       'opp_pass_compl_percentage', 'scorable_snaps', 'avg_scorable_snaps',
       'season_completion_percentage', 'home_away_flag', 'flag_a_turf',
 

# Interactive Visualization
***Sample Data***

In [88]:
import os
import random
currents = os.getcwd()
print(currents)

c:\Users\kenny\Team-134-CSE-6242-Project


In [ ]:
from bokeh import plotting, layouts, palettes
from bokeh.io import save, show, output_file
from bokeh.models import CustomJS, DataTable, DateFormatter, ColumnDataSource, Div, Select, TableColumn

def visualize_fantasy(file,predictor_variables_list = None):
    # Read the model output file
    df = pd.read_csv(file)
    # path to write output Frontend
    output_file("viz.html")
    # define needed components to visualize
    Layout,Figure,row,column,Viridis = layouts.layout,plotting.figure,plotting.row,plotting.column,palettes.Viridis256
    # Layout = layouts.layout,Figure = plotting.figurerow = plotting.row, column = plotting.column, Viridis = palettes.Viridis256
    # print df
    print(df)
    df[['week', 'team']] = df[['week', 'team']].astype(str)
    if predictor_variables_list:
        df = df[df['player_id'].isin(predictor_variables_list)]
    colors = random.sample(Viridis, len(df))
    df['color'] = colors
    # filter dataframe
    df1 = df[(df['week'] == '1-10') & (df['team'] == 'ATL')]
    # Column Data Source for plot and interactive callback reference
    src = ColumnDataSource(data=df)
    stateChange = ColumnDataSource(data=df1)

    # dropdown data list
    sort_df1 = df['week'].astype(str).unique().tolist()
    sort_df2 = df['team'].astype(str).unique().tolist()

    # default value for dropdown
    default_week = df.week[0]
    default_team = df.team[0]
    # key predictor variables/features
    variable_list = sorted(list(set(df['player_id'])))

    # code for javascript callback
    code_js = """
        var s1 = s1.value
        var s2 = s2.value
        sc.data['player_id']=[];
        sc.data['player_name']=[];
        sc.data['y_predicted']=[];
        sc.data['color']=[];    
        for (var i = 0; i < source.get_length(); i++) {
            if (source.data['week'][i] == s1 && source.data['team'][i] == s2) {
              sc.data['player_id'].push(source.data['player_id'][i]);
              sc.data['player_name'].push(source.data['player_name'][i]);
              sc.data['y_predicted'].push(source.data['y_predicted'][i]);
              sc.data['color'].push(source.data['color'][i]);
              sc.data['team'].push(source.data['team'][i]);
        }
      }
      sc.change.emit(); 
    """

    # dropdown
    select1 = Select(options= sort_df1, value=default_week, title = 'Week')
    select2 = Select(options= sort_df2, value=default_team, title = 'Team')
    # callback function for interactivity between plot,Data table and select/drop-down
    callback = CustomJS(args=dict(source = src, sc =stateChange, s1=select1, s2=select2), code= code_js) 


    plot=Figure(x_range=(0,10), x_axis_label = 'Fantasy Points', y_range=variable_list, y_axis_label = 'Players stats', height=600, width=700)

    # plot bar chart glyph
    plot.hbar('player_id', right='y_predicted', height=0.4, color='color', source=stateChange)
    plot.xgrid.grid_line_dash = 'dashed'
    plot.ygrid.grid_line_dash = 'dotted'

    # calling the function on change of selection
    select1.js_on_change('value', callback)
    select2.js_on_change('value', callback)

    # row for the dropdown
    control = row(select1, select2)

    # header for the layout
    div = Div(text="""<h1 style="background:lavender;font-size:20px; color:royalblue;">Fantasy Points Visualization</h1>
                      <h2 style="background:lavender;font-
                      size:15px; color:royalblue;">Seasons: 2015 - 2019 (live data)</h2>""",
                          sizing_mode="fixed", height=100, width=350)

    # header for chart
    div1 = Div(text="""<h2 style="font-size:15px; color:black;">Horizontal Bar Plot with Fantasy Model key's predictor variables</h2>""",
                          sizing_mode="fixed", height=50, width=500)

    # header for the data table
    div2 = Div(text="""<h2 style="font-size:15px; color:black;">Data</h2>""",
                          sizing_mode="fixed", height=50, width=350)

    # column for data table
    columns = [
            TableColumn(field="player_name", title="Players_name"),
            TableColumn(field="player_id", title="Players_id"),
            TableColumn(field="y_predicted", title="Fantasy Points"),
        ]

    # data table
    dtab = DataTable(source=stateChange, columns=columns, width=300, height=400)

    # creating column for layout
    col1=column(div, control, div2, dtab)
    # layout
    layout=row(col1, column(div1,plot))
    # save layout to output html file
    save(layout)
    pass
    # show(layout)
  

In [99]:
visualize_fantasy('sample_data/sample1.csv')

   player_id player_name  y_predicted  week team
0         gh         joe          2.0  1-10  ATL
1         es        jack          1.0  1-10  ATL
2          a       ethan          4.8  4-14  ATL
3          b       kenny          3.0  1-10  ATL
4          c     Bradley          2.0  1-10  ATL
5          d        liam          4.0  6-12  ATL
6          e      dexter          5.0  4-19  ATL
7          f        armi          6.2  3-15  ATL
8          g       gauge          5.3  2-14  ATL
9          h      parker          3.6   1-3  NYJ
10         i   christian          3.0  3-15  NYJ
11         j      julian          5.0  3-15  NYJ
12         k      carlos          4.9  6-12  NYJ
13         l      franco          6.0  1-10  NYJ
14         m        luke          3.0  4-14  NYJ
15         n    joe gage          2.0   1-5  LVR
16         o     jackson          4.0  4-19  LVR
17         p     brendan          6.8  3-15  LVR
18         q     holster          5.0  2-14  LVR
19         r      jo

***Bridged Data (In Progress)***

In [ ]:
# from bokeh.models import Dropdown, CustomJS, Column
# from bokeh.layouts import column
# from bokeh.plotting import show

# # Create the first dropdown
# dropdown1 = Dropdown(label="Dropdown 1", menu=[("Option 1", "1"), ("Option 2", "2")])
# output_file("dropdowns.html")
# # Create the second dropdown, initially empty
# dropdown2 = Dropdown(label="Dropdown 2")

# # Define the callback function
# callback = CustomJS(args=dict(dropdown1=dropdown1, dropdown2=dropdown2), code="""
#     const selectedValue = dropdown1.value;
#     const options = [];

#     if (selectedValue === "1") {
#         options.push(["Option A", "a"]);
#         options.push(["Option B", "b"]);
#     } else if (selectedValue === "2") {
#         options.push(["Option X", "x"]);
#         options.push(["Option Y", "y"]);
#     }

#     dropdown2.menu = options;
# """)

# # Set the callback on the first dropdown
# dropdown1.js_on_change('value', callback)

# # Layout the dropdowns
# layout = column(dropdown1, dropdown2)
# save(layout)
# # Show the plot
# show(layout)